# Building a database for crime reports

In this project, we will build a database for storing data related to crimes that occurred in Boston.

The goal of this project is to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file. We will be creating the table inside a schema named crimes. We will also create the readonly and readwrite groups with the appropriate privileges. Finally, we will also create one user for each of these groups.

## Initial exploration

Here are the first few rows:

In [27]:
import pandas as pd
import csv

In [28]:
#header and first four rows
pd.read_csv('boston.csv')[0:5]

,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date when the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with latitude and longitude coordinates.

## Creating the database and the schema

Let's create a database named crime_db for storing our crime data as well as a schema named crimes for containing the tables.

When connecting to the Postgres server, we need to specify a database name. Since the crime_db doesn't exist yet, we can create it by connecting to the dq database.

In [29]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
# set autocommit to True bacause this is required for creating databases
conn.autocommit = True
cur = conn.cursor()
# create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

ProgrammingError: database "crime_db" already exists


In [30]:
# now the crime_db database exists to we can connect to it
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()
# create he crimes schema
cur.execute("CREATE SCHEMA crimes;")

ProgrammingError: schema "crimes" already exists


We now have a database and a schema — we're ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can easily select the right datatypes to use in our table.

## Obtaining the Column Names and the first row

Let's start by reading the column names from the boston.csv file as well as the first row. This way, we will have them at hand throughout this project so we can easily refer to them.

We obtain the header row and assign it to a variable named col_headers and obtain the first data row and assign it to a variable named first_row.

In [31]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [32]:
print(col_headers)
print('\n')
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We now have access to the column headers and the first row of data. This will help us when we create the table. We won't have to remember the names of the columns, and using the first row of data, we can easily recall what kind of data is in each column.

## Creating a function for analyzing column values

Before we create a table to store the crime data, we need to identify the proper datatypes for the columns.

To help us with that, let's create a function — get_col_set() — that, given the name of a CSV file and a column index, computes a Python set with all distinct values contained in that column.

In [33]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


This function will be useful for two reasons:

- Checking whether an enumerated datatype might be a good choice for representing a column. Columns with a limited set of possible values are good candidates for enumerated datatypes.
- Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

There are two textual columns in the dataset, namely, the description and day_of_the_week columns. However, the day of the week contains only seven values — one for each day. We can tell that the longest of them is Wednesday without any computation.

Let's compute the maximum length of each value in the description column.

## Analyzing the maximum length of the description column

Use the get_col_set function to compute the maximum description length to decide an appropriate length for that field.

In [34]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [35]:
descriptions = get_col_set("boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the table

We will use the same names for the column headers.

In [36]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


From the result of printing first_row we see that kind of data that we have are:
- integer numbers
- integer numbers
- string
- date
- string
- decimal number
- decimal number

The number of different values of each column was:
- incident_number 298329
- offense_code       219
- description        239
- date              1177
- day_of_the_week      7
- lat              18177
- long             18177

In [37]:
# create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

We have created an enumerated datatype named weekday for the day_of_the_week since there there only seven possible values.

For the incident_number we have decided to use the type INTEGER and set it as the primary key. We also used same datatype INTEGER to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude we used DECIMAL datatypes.

## Load the data into the table

We use the copy_expert to load the data as it is very fast and very succinct to use.

In [38]:
# load the data from boston.csv into the table boston_crimes that is in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Revoke public privileges

Our goal is to create the two user groups: readonly and readwrite. 

By following the least privilege principle, as the first step we make sure that there are no privileges inherited from the public group and on the public schema.

In [39]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only group

The readonly group is supposed to only have privileges to perform SELECT queries.

We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema. Then we deal with tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema.

In [40]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

## Creating the read-write group

We want the readwrite group to be able to perform SELECT, INSERT, DELETE, and UPDATE queries. 

We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema. Then we deal with tables privileges by granting SELECT, INSERT, UPDATE and DELETE.

In [41]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating one user for each group

We create a user named data_analyst with password secret1 in the readonly group.

We create a user named data_scientist with password secret2 in the readwrite group.

In [42]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup

Let's test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [43]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
